# Projeto: **Recomendação de Produtos de Financiamentos do BNDES**


- O Banco Nacional de Desenvolvimento Econômico e Social (BNDES) é uma empresa pública federal com sede no Rio de Janeiro, cujo principal objetivo é o financiamento de *longo prazo* e investimento em todos os segmentos da economia brasileira.

- Problema: o banco deseja fazer uma prospecção ativa com o auxílio de um modelo para predizer o produto de financiamento que mais combina com o perfil do cliente.

OBS: Na prospecção ativa, chegamos até o potencial cliente sem que ele tenha solicitado um contato ou fornecido seus dados. Por outro lado, na prospecção passiva, o lead entra em contato com a sua organização ou fornece seus dados de forma espontânea, por conta própria.

- Na tentativa de facilitar a tomada de decisão sobre qual abordagem o time de marketing precisa utilizar, o modelo irá indicar, por faixas, as propensões que cada produto tem de ser o melhor para aquele perfil em análise.



# Etapa 1: Consolidação dos dados

O BNDES possui um banco de dados aberto em que, lá, temos o acesso ao download de dois arquivos de microdados de operações financeiras: operações indiretas automáticas e operações não automáticas. 

*Neste segundo arquivo contém tanto as operações diretas (contratadas diretamente com o BNDES) quanto as indiretas (contratadas através de agentes financeiros). As não automáticas são as que 2 necessitam de análise de um Analista do BNDES, e geralmente são as operações de valores maiores.

Link para download dos dois arquivos csv contendo os microdados:
- https://dadosabertos.bndes.gov.br/dataset/operacoes-financiamento/resource/9534f677-9525-4bf8-a3aa-fd5d3e152a93
- https://dadosabertos.bndes.gov.br/dataset/operacoes-financiamento/resource/332d446e-d340-46ef-af64-ee6f36e7bd50


## Importando os dados e vendo seu escopo

In [4]:
import pandas as pd 

In [ ]:
# df de operações automáticas
# pelo tamanho do df, ele demora um pouco para importar por completo
url1 = 'https://dadosabertos.bndes.gov.br/dataset/0f335c85-92a8-4343-9423-f073fb40774e/resource/9534f677-9525-4bf8-a3aa-fd5d3e152a93/download/operacoes-financiamento-operacoes-indiretas-automaticas.csv'

automaticas = pd.read_csv(url1, sep=';',
    decimal=',',
    na_values = '----------'
)
automaticas.tail(3)

In [ ]:
# vendo o tipo de cada variável do df
#automaticas.dtypes
automaticas.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105395 entries, 0 to 2105394
Data columns (total 30 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   cliente                             object 
 1   cpf_cnpj                            object 
 2   uf                                  object 
 3   municipio                           object 
 4   municipio_codigo                    int64  
 5   data_da_contratacao                 object 
 6   valor_da_operacao_em_reais          int64  
 7   valor_desembolsado_reais            float64
 8   fonte_de_recurso_desembolsos        object 
 9   custo_financeiro                    object 
 10  juros                               float64
 11  prazo_carencia_meses                int64  
 12  prazo_amortizacao_meses             int64  
 13  modalidade_de_apoio                 object 
 14  forma_de_apoio                      object 
 15  produto                             object 
 16  

# Seleção e filtro de dados

Aqui, temos dois caminhos que podem ser seguidos. Se consideramos a variável "valor da operação", seguiremos para um modelo para prospecção **passiva**, mas, se não levamos em consideração tal variável, seguiremos para um modelo de prospecção **ativa**. Nessa perspectiva, seguiremos com o último caminho mencionado.

In [ ]:
#caminho caso utilizassemos essa variável
#operacoes = automaticas[['valor_da_operacao_em_reais','data_da_contratacao','uf','natureza_do_cliente','porte_do_cliente','subsetor_bndes','inovacao','produto']]

#caminho sem essa variável
operacoes = automaticas[['data_da_contratacao','uf','natureza_do_cliente','porte_do_cliente','subsetor_bndes','inovacao','produto']]

operacoes

,data_da_contratacao,uf,natureza_do_cliente,porte_do_cliente,subsetor_bndes,inovacao,produto
0,2002-01-02,RS,PRIVADA,PEQUENA,OUTROS TRANSPORTES,NÃO,BNDES AUTOMÁTICO
1,2002-01-02,RS,PRIVADA,MICRO,AGROPECUÁRIA,NÃO,BNDES AUTOMÁTICO
2,2002-01-02,RS,PRIVADA,MICRO,AGROPECUÁRIA,NÃO,BNDES AUTOMÁTICO
3,2002-01-02,RS,PRIVADA,MICRO,AGROPECUÁRIA,NÃO,BNDES AUTOMÁTICO
4,2002-01-02,RS,PRIVADA,MICRO,AGROPECUÁRIA,NÃO,BNDES AUTOMÁTICO
...,...,...,...,...,...,...,...
2105390,2021-12-28,RN,PRIVADA,PEQUENA,COMÉRCIO E SERVIÇOS,NÃO,BNDES AUTOMÁTICO
2105391,2021-12-28,MA,PRIVADA,MÉDIA,TRANSPORTE RODOVIÁRIO,NÃO,BNDES FINAME
2105392,2021-12-28,RS,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES FINAME
2105393,2021-12-28,PR,PRIVADA,MÉDIA,TRANSPORTE RODOVIÁRIO,NÃO,BNDES FINAME


# Pré-processamento dos Dados

## Filtrando somente os últimos 4 anos

Aqui, vale a pena a reflexão sobre valeria a pena pegarmos dados muito antigos!

In [ ]:
#Criando a variavel ano
operacoes['ano'] = operacoes['data_da_contratacao'].apply(lambda x: int(x[0:4]))
operacoes['ano'].value_counts()

2011    237983
2010    209476
2012    208995
2013    193584
2014    172939
2009    140111
2008    111238
2007     97439
2004     87546
2015     86293
2005     85206
2006     69429
2002     65893
2003     65479
2017     61368
2016     52833
2020     46707
2018     46462
2021     34965
2019     31449
Name: ano, dtype: int64

In [ ]:
operacoes['data_da_contratacao']

0          2002-01-02
1          2002-01-02
2          2002-01-02
3          2002-01-02
4          2002-01-02
              ...    
2105390    2021-12-28
2105391    2021-12-28
2105392    2021-12-28
2105393    2021-12-28
2105394    2021-12-28
Name: data_da_contratacao, Length: 2105395, dtype: object

In [ ]:
# Peganndo os últimos 4 anos e verificando a quantidade que temos pra cada produto
operacoes.loc[(operacoes['ano'] > 2017)]['produto'].value_counts()

BNDES FINAME        81199
BNDES AUTOMÁTICO    78384
Name: produto, dtype: int64

In [ ]:
operacoes = operacoes.loc[(operacoes['ano'] > 2017)]
operacoes

,data_da_contratacao,uf,natureza_do_cliente,porte_do_cliente,subsetor_bndes,inovacao,produto,ano
1945812,2018-01-02,SC,PRIVADA,MICRO,COMÉRCIO E SERVIÇOS,NÃO,BNDES AUTOMÁTICO,2018
1945813,2018-01-02,ES,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES FINAME,2018
1945814,2018-01-02,SP,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES FINAME,2018
1945815,2018-01-02,MT,PRIVADA,MÉDIA,ALIMENTO E BEBIDA,NÃO,BNDES FINAME,2018
1945816,2018-01-02,SP,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES AUTOMÁTICO,2018
...,...,...,...,...,...,...,...,...
2105390,2021-12-28,RN,PRIVADA,PEQUENA,COMÉRCIO E SERVIÇOS,NÃO,BNDES AUTOMÁTICO,2021
2105391,2021-12-28,MA,PRIVADA,MÉDIA,TRANSPORTE RODOVIÁRIO,NÃO,BNDES FINAME,2021
2105392,2021-12-28,RS,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES FINAME,2021
2105393,2021-12-28,PR,PRIVADA,MÉDIA,TRANSPORTE RODOVIÁRIO,NÃO,BNDES FINAME,2021


In [ ]:
# drop de variáveis indesejadas
operacoes = operacoes.drop(['data_da_contratacao', 'ano'], axis=1)
operacoes

,uf,natureza_do_cliente,porte_do_cliente,subsetor_bndes,inovacao,produto
1945812,SC,PRIVADA,MICRO,COMÉRCIO E SERVIÇOS,NÃO,BNDES AUTOMÁTICO
1945813,ES,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES FINAME
1945814,SP,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES FINAME
1945815,MT,PRIVADA,MÉDIA,ALIMENTO E BEBIDA,NÃO,BNDES FINAME
1945816,SP,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES AUTOMÁTICO
...,...,...,...,...,...,...
2105390,RN,PRIVADA,PEQUENA,COMÉRCIO E SERVIÇOS,NÃO,BNDES AUTOMÁTICO
2105391,MA,PRIVADA,MÉDIA,TRANSPORTE RODOVIÁRIO,NÃO,BNDES FINAME
2105392,RS,PRIVADA,MÉDIA,COMÉRCIO E SERVIÇOS,NÃO,BNDES FINAME
2105393,PR,PRIVADA,MÉDIA,TRANSPORTE RODOVIÁRIO,NÃO,BNDES FINAME


## Checando a consistência das variáveis


In [ ]:
#verificando os "balanceamento" da variável UF
operacoes['uf'].value_counts()

 SP    31902
 PR    25508
 SC    20189
 MG    19577
 RS    18535
 MT     7092
 GO     5915
 RJ     5011
 BA     4889
 MS     3246
 ES     2948
 PA     2335
 PE     1727
 MA     1622
 CE     1511
 RO     1287
 TO     1137
 DF     1011
 AM      872
 PB      710
 RN      597
 PI      577
 AL      434
 SE      331
 RR      239
 AC      229
 AP      152
Name: uf, dtype: int64

In [ ]:
operacoes['natureza_do_cliente'].value_counts()

PRIVADA                                             159459
ADMINISTRAÇÃO PÚBLICA DIRETA - GOVERNO MUNICIPAL       110
PÚBLICA INDIRETA                                        14
Name: natureza_do_cliente, dtype: int64

In [ ]:
#Redesenhando a variavel e verificando o balanceamento delas
def aux(x):
    return 'PÚBLICA' if x != 'PRIVADA' else x
    
operacoes['natureza_do_cliente'] = operacoes['natureza_do_cliente'].apply(aux)
operacoes['natureza_do_cliente'].value_counts()

PRIVADA    159459
PÚBLICA       124
Name: natureza_do_cliente, dtype: int64

In [ ]:
operacoes['porte_do_cliente'].value_counts()

PEQUENA    56460
MÉDIA      48703
MICRO      34875
GRANDE     19545
Name: porte_do_cliente, dtype: int64

In [ ]:
operacoes['subsetor_bndes'].value_counts()

COMÉRCIO E SERVIÇOS        83439
TRANSPORTE RODOVIÁRIO      44955
OUTRAS                      6990
ALIMENTO E BEBIDA           6052
AGROPECUÁRIA                5651
METALURGIA E PRODUTOS       2238
MECÂNICA                    1874
TÊXTIL E VESTUÁRIO          1726
QUÍMICA E PETROQUÍMICA      1341
SERV. UTILIDADE PÚBLICA     1104
ATV. AUX. TRANSPORTES       1074
EXTRATIVA                    721
CELULOSE E PAPEL             626
TELECOMUNICAÇÕES             562
MATERIAL DE TRANSPORTE       561
ENERGIA ELÉTRICA             268
CONSTRUÇÃO                   243
OUTROS                        97
OUTROS TRANSPORTES            57
TRANSPORTE FERROVIÁRIO         4
Name: subsetor_bndes, dtype: int64

In [ ]:
# Padronizando os níveis de fatores da subsetor_bndes
operacoes['subsetor_bndes'] = operacoes['subsetor_bndes'].replace(
    ['OUTRAS'],['OUTROS'])
operacoes['subsetor_bndes'].unique()

array(['COMÉRCIO E SERVIÇOS', 'ALIMENTO E BEBIDA',
       'TRANSPORTE RODOVIÁRIO', 'TELECOMUNICAÇÕES', 'CELULOSE E PAPEL',
       'OUTROS', 'SERV. UTILIDADE PÚBLICA', 'METALURGIA E PRODUTOS',
       'ATV. AUX. TRANSPORTES', 'AGROPECUÁRIA', 'EXTRATIVA',
       'QUÍMICA E PETROQUÍMICA', 'ENERGIA ELÉTRICA', 'TÊXTIL E VESTUÁRIO',
       'MECÂNICA', 'MATERIAL DE TRANSPORTE', 'CONSTRUÇÃO',
       'OUTROS TRANSPORTES', 'TRANSPORTE FERROVIÁRIO'], dtype=object)

In [ ]:
operacoes['inovacao'].value_counts()	

NÃO    159467
SIM       116
Name: inovacao, dtype: int64

In [ ]:
operacoes['produto'].value_counts()

# Hands on para o ML

Neste modelo, vamos utilizar o PyCaret. O PyCaret é uma biblioteca de machine learning simples, fácil de aprender e de low-code em Python.


VALIDAÇÃO: Default do PyCaret - Holdout + KFold.

A métrica que servirá de embasamento será o F1-Score - média harmônica entre a recall e a precisão.

* Recall (ou sensibilidade): o quanto que acertou dos positivos

* Precisão: dado que meu modelo fala que é X, qual a probabilidade ele estar certo?


In [ ]:
# como ainda estamos em um nível experimental, pegaremos uma amostra de 5k.
n = 5000
operacoes_amostra = operacoes.sample(n)

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import setup, models, create_model, tune_model, \
                                   compare_models, blend_models, stack_models, \
                                   predict_model, evaluate_model, plot_model, \
                                   finalize_model, save_model, load_model, optimize_threshold

In [ ]:
#dados = operacoes
dados = operacoes_amostra

classificacao = setup(data = dados, 
                      target = 'produto', 
                      normalize = True,
                      combine_rare_levels = True, #deixando em true pois, quando criar uma variável com pouca variância, ele combina a variável
                      ignore_low_variance = True, #este tem a mesma função, mas no caso, ele exclui aquelas com pouca variância
                      # bin_numeric_features = ['valor_da_operacao_em_reais'] --> se eu tivesse incluindo a variável "valor da operação"
                      # profile = True --> caso eu quisesse gerar o pandas profiling, biblioteca de análise exploratória, assim como o sweetviz)
                      )

# aqui ocorre uma estratégia de hold out combinado com kfold. No "Transformed train set"e "transformed test set" temos o hold out deixando
# en uma perspectiva 70% para treino e 30% teste. Já o KFold, utilizamos só para os dados de treino, porque aí, serve de setup para analisar
# as duas grandes necessidades de validação: escolha do modelo mais satisfatório, e estimar o quanto que o modelo escolhido vai errar na prática!

,Description,Value
0,session_id,1235
1,Target,produto
2,Target Type,Binary
3,Label Encoded,"BNDES AUTOMÁTICO: 0, BNDES FINAME: 1"
4,Original Data,"(5000, 6)"
5,Missing Values,False
6,Numeric Features,0
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
#modelos = ['lr', 'dt', 'rf', 'et', 'nb', 'mlp', 'rbfsvm', 'lightgbm', 'gbc', 'ada']

# estes aqui foram os escolhidos por representar modelos de naturezas distintas
modelos = ['lr', 'dt', 'rf', 'et', 'nb', 'lightgbm', 'gbc', 'ada']

melhores5 = compare_models(include = modelos, n_select = 5, sort = 'F1')


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8622,0.9306,0.8787,0.8551,0.8665,0.7243,0.7249,0.315
lightgbm,Light Gradient Boosting Machine,0.8634,0.9280,0.8635,0.8678,0.8654,0.7267,0.7270,0.117
rf,Random Forest Classifier,0.8608,0.9203,0.8596,0.8664,0.8627,0.7216,0.7220,0.697
lr,Logistic Regression,0.8588,0.9288,0.8702,0.8556,0.8626,0.7175,0.7181,0.497
et,Extra Trees Classifier,0.8582,0.9056,0.8472,0.8712,0.8588,0.7165,0.7172,0.586
ada,Ada Boost Classifier,0.8548,0.9262,0.8612,0.8552,0.8578,0.7095,0.7101,0.193
dt,Decision Tree Classifier,0.8551,0.8935,0.8438,0.8684,0.8556,0.7103,0.7110,0.027
nb,Naive Bayes,0.7596,0.8681,0.6500,0.8397,0.7268,0.5213,0.5380,0.020


In [ ]:
modelo = create_model('lightgbm')
lightgbm = tune_model(modelo, optimize = 'F1', n_iter = 20, choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8571,0.9113,0.8652,0.8556,0.8603,0.7141,0.7142
1,0.8714,0.9323,0.8820,0.8674,0.8747,0.7427,0.7428
2,0.8543,0.9230,0.9101,0.8223,0.8640,0.7079,0.7122
3,0.8457,0.9253,0.8315,0.8605,0.8457,0.6915,0.6919
4,0.8343,0.9148,0.8539,0.8261,0.8398,0.6683,0.6687
5,0.8686,0.9310,0.8652,0.8750,0.8701,0.7371,0.7372
6,0.8886,0.9465,0.8820,0.8971,0.8895,0.7771,0.7773
7,0.8486,0.9307,0.8764,0.8342,0.8548,0.6968,0.6977
8,0.8943,0.9476,0.8764,0.9123,0.8940,0.7887,0.7893
9,0.8682,0.9326,0.8596,0.8793,0.8693,0.7364,0.7366


In [ ]:
modelo = create_model('gbc')
gbc_tunado = tune_model(modelo, optimize = 'F1', n_iter = 20, choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8514,0.9093,0.8483,0.8580,0.8531,0.7028,0.7029
1,0.8771,0.9352,0.8933,0.8689,0.8809,0.7541,0.7544
2,0.8457,0.9232,0.8933,0.8196,0.8548,0.6909,0.6938
3,0.8457,0.9248,0.8371,0.8563,0.8466,0.6915,0.6916
4,0.8343,0.9174,0.8539,0.8261,0.8398,0.6683,0.6687
5,0.8657,0.9322,0.8596,0.8743,0.8669,0.7314,0.7315
6,0.8914,0.9485,0.8876,0.8977,0.8927,0.7828,0.7829
7,0.8629,0.9309,0.8876,0.8495,0.8681,0.7254,0.7262
8,0.9000,0.9503,0.8820,0.9181,0.8997,0.8001,0.8007
9,0.8596,0.9302,0.8708,0.8564,0.8635,0.7190,0.7191


In [ ]:
modelo = create_model('rf')
rf_tunado = tune_model(modelo, optimize = 'F1', n_iter = 10, choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8514,0.9058,0.8427,0.8621,0.8523,0.7029,0.7031
1,0.8714,0.9352,0.8933,0.8595,0.8760,0.7426,0.7432
2,0.8371,0.9188,0.8764,0.8168,0.8455,0.6738,0.6756
3,0.8457,0.9223,0.8315,0.8605,0.8457,0.6915,0.6919
4,0.8314,0.9199,0.8539,0.8216,0.8375,0.6625,0.6631
5,0.8657,0.9298,0.8539,0.8786,0.8661,0.7315,0.7318
6,0.8886,0.9484,0.8764,0.9017,0.8889,0.7772,0.7775
7,0.8714,0.9277,0.8933,0.8595,0.8760,0.7426,0.7432
8,0.8971,0.9463,0.8876,0.9080,0.8977,0.7943,0.7945
9,0.8711,0.9354,0.8764,0.8715,0.8739,0.7420,0.7420


In [ ]:
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8286,0.9097,0.8315,0.8315,0.8315,0.6570,0.6570
1,0.8686,0.9322,0.8876,0.8587,0.8729,0.7369,0.7373
2,0.8429,0.9240,0.8933,0.8154,0.8525,0.6851,0.6884
3,0.8371,0.9202,0.8371,0.8418,0.8394,0.6742,0.6742
4,0.8400,0.9152,0.8539,0.8352,0.8444,0.6798,0.6800
5,0.8686,0.9298,0.8708,0.8708,0.8708,0.7371,0.7371
6,0.8800,0.9478,0.8876,0.8778,0.8827,0.7599,0.7599
7,0.8543,0.9308,0.8876,0.8360,0.8610,0.7082,0.7096
8,0.9000,0.9483,0.8820,0.9181,0.8997,0.8001,0.8007
9,0.8682,0.9304,0.8708,0.8708,0.8708,0.7363,0.7363


In [ ]:
# método de ajudar a optimizar o ponto de corte, consequentemente, ajudar a otimizar o modelo
lr_ot = optimize_threshold(lr)

In [ ]:
# pegando os 5 melhores modelos e fazendo uma combinação soft
# soft = média das propensões e faz a classificação
# hard = pega os votos de cada modelo, e a maioria dos votos gera o voto final que é o modelo combinado

modelo_combinado = blend_models(melhores5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8486,0.9086,0.8427,0.8571,0.8499,0.6971,0.6972
1,0.8714,0.9324,0.8764,0.8715,0.8739,0.7428,0.7428
2,0.8457,0.9197,0.8876,0.8229,0.8541,0.6909,0.6931
3,0.8486,0.9226,0.8371,0.8613,0.8490,0.6972,0.6975
4,0.8286,0.9141,0.8427,0.8242,0.8333,0.6569,0.6571
5,0.8714,0.9349,0.8539,0.8889,0.8711,0.7430,0.7436
6,0.8886,0.9472,0.8820,0.8971,0.8895,0.7771,0.7773
7,0.8657,0.9272,0.8820,0.8579,0.8698,0.7312,0.7315
8,0.8943,0.9480,0.8820,0.9075,0.8946,0.7886,0.7889
9,0.8711,0.9351,0.8708,0.8757,0.8732,0.7420,0.7421


In [ ]:
# combinando os modelos, utiliza as propensões como variáveis e deixa o modelo fazer a propensão final. Assim, um outro modelo a parte
# vai combinar esse modelos como variáveis explicativas da variável resposta final

modelo_stack = stack_models(melhores5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8371,0.9105,0.8202,0.8538,0.8367,0.6744,0.6750
1,0.8743,0.9336,0.8876,0.8681,0.8778,0.7484,0.7486
2,0.8457,0.9245,0.8933,0.8196,0.8548,0.6909,0.6938
3,0.8400,0.9225,0.8427,0.8427,0.8427,0.6799,0.6799
4,0.8400,0.9162,0.8427,0.8427,0.8427,0.6799,0.6799
5,0.8714,0.9316,0.8652,0.8800,0.8725,0.7429,0.7430
6,0.8914,0.9494,0.8876,0.8977,0.8927,0.7828,0.7829
7,0.8600,0.9318,0.8933,0.8413,0.8665,0.7196,0.7210
8,0.9029,0.9486,0.8876,0.9186,0.9029,0.8058,0.8062
9,0.8739,0.9323,0.8764,0.8764,0.8764,0.7477,0.7477


## Escolha do Modelo


In [ ]:
# gbc tunado deu 0.8666
modelo_de_trabalho = gbc_tunado

In [ ]:
evaluate_model(modelo_de_trabalho)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
# Análise do Modelo
plot_model(modelo_de_trabalho, 'feature_all', save = True)
plot_model(modelo_de_trabalho, 'auc', save = True)
plot_model(modelo_de_trabalho, 'confusion_matrix', save = True) 

'Confusion Matrix.png'

In [ ]:
predicoes

,uf_ AL,uf_ AM,uf_ BA,uf_ CE,uf_ DF,uf_ ES,uf_ GO,uf_ MA,uf_ MG,uf_ MS,...,subsetor_bndes_QUÍMICA E PETROQUÍMICA,subsetor_bndes_SERV. UTILIDADE PÚBLICA,subsetor_bndes_TELECOMUNICAÇÕES,subsetor_bndes_TRANSPORTE RODOVIÁRIO,subsetor_bndes_TÊXTIL E VESTUÁRIO,subsetor_bndes_others_infrequent,produto,Label,Score_BNDES AUTOMÁTICO,Score_BNDES FINAME
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.8318,0.1682
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.9663,0.0337
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES AUTOMÁTICO,0.8994,0.1006
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES FINAME,0.0268,0.9732
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES AUTOMÁTICO,0.7960,0.2040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.8710,0.1290
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,BNDES FINAME,BNDES FINAME,0.0255,0.9745
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.9210,0.0790
1499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES AUTOMÁTICO,0.5438,0.4562


# Criando as faixas a partir das predições

In [ ]:
predicoes['faixa'] = pd.qcut(predicoes['Score_BNDES FINAME'], 5, labels = list('EDCBA'))
predicoes

,uf_ AL,uf_ AM,uf_ BA,uf_ CE,uf_ DF,uf_ ES,uf_ GO,uf_ MA,uf_ MG,uf_ MS,...,subsetor_bndes_SERV. UTILIDADE PÚBLICA,subsetor_bndes_TELECOMUNICAÇÕES,subsetor_bndes_TRANSPORTE RODOVIÁRIO,subsetor_bndes_TÊXTIL E VESTUÁRIO,subsetor_bndes_others_infrequent,produto,Label,Score_BNDES AUTOMÁTICO,Score_BNDES FINAME,faixa
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.8318,0.1682,D
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.9663,0.0337,E
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES AUTOMÁTICO,0.8994,0.1006,D
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES FINAME,0.0268,0.9732,A
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES AUTOMÁTICO,0.7960,0.2040,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.8710,0.1290,D
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,BNDES FINAME,BNDES FINAME,0.0255,0.9745,A
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES AUTOMÁTICO,BNDES AUTOMÁTICO,0.9210,0.0790,E
1499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,BNDES FINAME,BNDES AUTOMÁTICO,0.5438,0.4562,C


In [ ]:
predicoes['faixa'].value_counts()

E    311
C    306
B    299
A    294
D    291
Name: faixa, dtype: int64

In [ ]:
# vendo as faixas
from sklearn.metrics import classification_report  

for faixa in list('ABCDE'):
    pred = predicoes[predicoes['faixa'] == faixa]
    y_pred = pred['Label']
    y_obs = pred['produto']
    print('\n*** Métricas do Modelo avaliado apenas na Faixa {} ***\n'.format(faixa))
    print(classification_report(y_obs, y_pred, target_names = ['AUTO', 'FINAME'], digits = 4, output_dict = False))
    print(53*'-')


*** Métricas do Modelo avaliado apenas na Faixa A ***

              precision    recall  f1-score   support

        AUTO     0.0000    0.0000    0.0000         3
      FINAME     0.9898    1.0000    0.9949       291

    accuracy                         0.9898       294
   macro avg     0.4949    0.5000    0.4974       294
weighted avg     0.9797    0.9898    0.9847       294

-----------------------------------------------------

*** Métricas do Modelo avaliado apenas na Faixa B ***

              precision    recall  f1-score   support

        AUTO     0.0000    0.0000    0.0000        51
      FINAME     0.8294    1.0000    0.9068       248

    accuracy                         0.8294       299
   macro avg     0.4147    0.5000    0.4534       299
weighted avg     0.6880    0.8294    0.7521       299

-----------------------------------------------------

*** Métricas do Modelo avaliado apenas na Faixa C ***

              precision    recall  f1-score   support

        AUTO   

In [ ]:
marketing = predicoes[['Score_BNDES FINAME', 'faixa']]

def aux(x):
    if x == 'A':
        return 'Recomendar FINAME - Muito Alta Confiança'
    elif x == 'B':
        return 'Recomendar FINAME - Alta Confiança'
    elif x == 'C':
        return 'Zona Cinza, Evitar Abordagem'
    elif x == 'D':
        return 'Recomendar AUTO - Alta Confiança'
    elif x == 'E':
        return 'Recomendar AUTO - Muito Alta Confiança'

marketing['Recomendação'] = marketing['faixa'].apply(aux)
marketing.sample(30)

,Score_BNDES FINAME,faixa,Recomendação
195,0.6349,C,"Zona Cinza, Evitar Abordagem"
1440,0.8940,B,Recomendar FINAME - Alta Confiança
770,0.8202,B,Recomendar FINAME - Alta Confiança
597,0.3183,C,"Zona Cinza, Evitar Abordagem"
540,0.8677,B,Recomendar FINAME - Alta Confiança
538,0.5657,C,"Zona Cinza, Evitar Abordagem"
372,0.1290,D,Recomendar AUTO - Alta Confiança
71,0.0337,E,Recomendar AUTO - Muito Alta Confiança
512,0.9475,B,Recomendar FINAME - Alta Confiança
520,0.7233,C,"Zona Cinza, Evitar Abordagem"


# deploy




In [ ]:
marketing.to_excel('para_o_pessoal_do_mkt.xlsx')

In [ ]:
modelo_final = finalize_model(modelo_de_trabalho)

In [ ]:
save_model(modelo_final, 'modelo-recomendacao-produtos-BNDES')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='produto',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_stra...
                                             learning_rate=0.05, loss='deviance',
                                             max_depth=7, max_features='log2',
                                             max_leaf_nodes=None,
                                             mi

A análise ainda não pararia por aqui, de uma forma user friendly, poderiamos usar o Streamlit para o time de marketing utilizar!

De toda forma, com o modelo gerado, conseguimos concluir nosso objetivo.